In [ ]:
from sodapy import Socrata
import json
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from config import appToken

from urllib.request import urlopen
from scipy import stats
import gmaps
import plotly.express as px



# Use open client to obtain data
client = Socrata("data.cdc.gov", appToken)
results = client.get("dnhi-s2bf", limit = 1000000)

# Convert to pandas DataFrame
death_quarterly = pd.DataFrame.from_records(results)
death_quarterly.head()


In [ ]:
death_quarterly_1 = death_quarterly.drop(columns=['footnote'])
death_quarterly_2 = death_quarterly_1.dropna(subset =['covid_19_deaths','total_deaths'])


In [ ]:
death_quarterly_2['total_deaths'] = death_quarterly_2['total_deaths'].astype(int)
death_quarterly_2['covid_19_deaths'] = death_quarterly_2['covid_19_deaths'].astype(int)
death_gb = death_quarterly_2.groupby(['fips_code'])
death_gb_df = death_gb.sum()

In [ ]:
death_gb_df['covid_portion_of_total_deaths'] = death_gb_df['covid_19_deaths']/death_gb_df['total_deaths']
death_gb_df = pd.DataFrame(death_gb_df)

In [ ]:
death_gb_df

In [ ]:
death_gb_df.reset_index(inplace=True)

In [ ]:
death_gb_df.describe(include='all')

In [ ]:
death_quarterly_2

In [ ]:
url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"

response = requests.get(url).json()
response

In [ ]:
fig = px.choropleth(death_gb_df, geojson=response, locations='fips_code', color='covid_portion_of_total_deaths',
                           color_continuous_scale="bluered",
                           range_color=(0.1, .25),
                           scope="usa",
                           labels={'COVID as % of Total Deaths':'covid_portion_of_total_deaths'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


#https://plotly.com/python/choropleth-maps/
#https://plotly.com/python/builtin-colorscales/